In [ ]:
import glob
import os
from pptx import Presentation
import xlsxwriter
from pptx.enum.action import PP_ACTION
from pptx.enum.shapes import MSO_SHAPE_TYPE

n=1
image_filename=""
def write_image(shape):
    global n
    global image_filename
    image = shape.image
    # ---get image "file" contents---
    image_bytes = image.blob
    # ---make up a name for the file, e.g. 'image.jpg'---
    image_filename = fileName_only + '_image{:03d}.{}'.format(n, image.ext)
    n += 1
    print(image_filename)
    with open(image_filename, 'wb') as f:
        f.write(image_bytes)

workbook = xlsxwriter.Workbook('hmotors_ad.xlsx')

for file in glob.glob("./*.pptx"):
    fileName_only = os.path.basename(file)

#     print(fileName_only)
    worksheet = workbook.add_worksheet(name = fileName_only)
    e_row = 0
    e_col = 0
    n=1

    prs = Presentation(file)

#     slides = prs.slides
#     for slide in slides:

    for slide in prs.slides:
        slide_num = prs.slides.index(slide)
        if(slide_num==0): 
            continue
        for shape in slide.shapes:
            if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                write_image(shape)            
                worksheet.write(e_row,e_col,image_filename)
                e_col += 1

            click_action = shape.click_action
            if click_action.action == PP_ACTION.HYPERLINK:        
                worksheet.write(e_row,e_col,click_action.hyperlink.address)
#                 print(e_row+e_col)
                e_row += 1
                e_col = 0   
            if not shape.has_table:
                continue    
            tbl = shape.table
            row_count = len(tbl.rows)
            col_count = len(tbl.columns)
            for r in range(1, row_count):            #첫줄에 공백이 있음 & 첫 슬라의 prepared for 제거
                for c in range(0, col_count):
                    cell = tbl.cell(r,c)
                    worksheet.write(e_row,e_col,cell.text)
                    e_col += 1
                e_row += 1
                e_col = 0    

        if(slide_num>0):                             #첫 슬라이드 제외
            e_row += 1
            
            
workbook.close()
    